In [10]:
import sys
sys.path.append("../SpikeCounter")
import numpy as np
import skimage.io as skio
import matplotlib as mpl
import matplotlib.pyplot as plt
from tifffile import imsave
import matplotlib.colors as colors
import matplotlib.patches as patches
import os
from scipy import interpolate, optimize, ndimage, stats, signal
from skimage import filters, transform, morphology, exposure, measure
from skimage import color as ski_color
from spikecounter.analysis import traces, images
from spikecounter.ui import visualize
from spikecounter.ui import stackViewer
from spikecounter import utils
import colorcet as cc
from cycler import cycler
from scipy.io import loadmat
import mat73
from sklearn.utils.extmath import randomized_svd
import pandas as pd
import importlib
from statannotations.Annotator import Annotator

import seaborn as sns
plt.style.use("/n/home11/bjia/bio_publications.mplstyle")

In [2]:
def reload_libraries():
    importlib.reload(images)
    importlib.reload(traces)
    importlib.reload(utils)
    importlib.reload(visualize)

In [3]:
figdir = "/n/cohen_lab/Lab/Papers/2022 First Heartbeat/Figures/"

In [4]:
um_per_px

NameError: name 'um_per_px' is not defined

### Load metadata and video

In [22]:
reload_libraries()

In [7]:
rootdir = "/n/holyscratch01/cohen_lab/bjia/20211124_rel_calcium_contraction/"
subfolder = "downsampled"
um_per_px = 0.265*4
# os.makedirs(os.path.join(rootdir, "analysis"), exist_ok=True)

In [37]:
um_per_px

1.06

In [5]:
pairs = {"before": ["122345E12_bf", "122202E12_calcium"], "after": ["133353E12_bf","133603E12_calcium"]}
img_pairs = {"before": [], "after": []}

In [8]:
for key, value in pairs.items():
    for f in value:
        
        img, expt_data = images.load_image(rootdir, f, subfolder=subfolder)
        traces_dict, t = utils.traces_to_dict(expt_data)
        dt = np.mean(np.diff(t))
        t = t[:img.shape[0]]
        print(dt)
        img_pairs[key].append(img)
# os.makedirs(os.path.join(rootdir, "analysis", subfolder, filename), exist_ok=True)

{'camera': {'dropped_frames': array(0.), 'exposuretime': array(0.10000638), 'frames_requested': array(600.), 'roi': array([672., 704., 672., 704.])}, 'clock_rate': array(100000.), 'dmd_fast': {'target': array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]]), 'target_image_space': array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]]), 'tform': array([[-2.18731416e-01, -2.14308304e-02,  0.00000000e+00],
      

/n/holyscratch01/cohen_lab/bjia/notebooks/../SpikeCounter/spikecounter/analysis/images.py:82: UserWarning: 9 frames dropped
  warnings.warn("%d frames dropped" % (fc_max - img.shape[0]))


0.10319083881578947
{'camera': {'dropped_frames': array(0.), 'exposuretime': array(0.10000638), 'frames_requested': array(600.), 'roi': array([672., 704., 672., 704.])}, 'clock_rate': array(100000.), 'dmd_fast': {'target': array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]]), 'target_image_space': array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]]), 'tform': array([[-2.18731416e-01, -2.14308304e-02,  0.0

In [33]:
key = "after"
imgs = img_pairs[key]
rgb_bf = np.moveaxis(ski_color.gray2rgb(exposure.rescale_intensity(imgs[0], out_range=np.uint8)), -1, 1)
rgb_calcium = np.moveaxis(np.array([np.zeros_like(imgs[1], dtype=np.uint8),\
                        exposure.rescale_intensity(imgs[1], out_range=np.uint8), \
                        np.zeros_like(imgs[1], dtype=np.uint8)]), 0, 1)

In [34]:
imgs[0].shape

(600, 352, 352)

In [35]:
combined_img = np.concatenate([rgb_bf, rgb_calcium], axis=3)

In [36]:
skio.imsave(os.path.join(rootdir, "after_contraction.tif"), combined_img)

In [30]:
combined_img.shape

(600, 3, 352, 704)